<a href="https://colab.research.google.com/github/Tclack88/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/LS_DS_432_Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/Tclack88/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/mountain/art1131.jpg?raw=1)|![](https://github.com/Tclack88/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/forest/cdmc317.jpg?raw=1)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [1]:
!ls

forest	mountain  Thumbs.db


In [0]:
!rm -r sample_data/

In [0]:
mkdir forest && mv *.jpg forest # Run after uploading forest data

In [0]:
mkdir mountain && mv *.jpg mountain # Run after uploading mountain data

## Instatiate Model

In [2]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
for layer in resnet.layers:
    layer.trainable = False

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [0]:
import os
forest_images = ['./forest/'+ image for image in os.listdir('./forest')]
mountain_images = ['./mountain/'+ image for image in os.listdir('./mountain')]

In [0]:
# Create data labels (y)
y0 = np.zeros(len(forest_images))
y1 = np.ones(len(mountain_images))
y = np.hstack((y0,y1))  # create labelled data (0 is forest, 1 is mountain)

In [0]:
# Create data arrays (X)
import PIL
X = []
for image in (forest_images + mountain_images):
  image_array = np.asarray(PIL.Image.open(image))
  X.append(image_array)

X = np.array(X) / 255 # normalize pixel values by dividing by 255

In [8]:
X.shape

(702, 256, 256, 3)

In [0]:
# Segment off some of the data to test later
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X,y, random_state=88)

## Fit Model

In [0]:
model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])

In [90]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Train on 526 samples, validate on 176 samples
Epoch 1/5
526/526 [==============================] - 4s 8ms/sample - loss: 0.6336 - acc: 0.8650 - val_loss: 0.7191 - val_acc: 0.5114
Epoch 2/5
526/526 [==============================] - 2s 4ms/sample - loss: 0.2302 - acc: 0.9297 - val_loss: 0.7453 - val_acc: 0.5114
Epoch 3/5
526/526 [==============================] - 2s 4ms/sample - loss: 0.0138 - acc: 0.9943 - val_loss: 1.0735 - val_acc: 0.5114
Epoch 4/5
526/526 [==============================] - 2s 4ms/sample - loss: 0.0121 - acc: 0.9981 - val_loss: 1.0351 - val_acc: 0.5114
Epoch 5/5
526/526 [==============================] - 2s 4ms/sample - loss: 0.0141 - acc: 0.9962 - val_loss: 1.2731 - val_acc: 0.5114


In [76]:
model.evaluate(X_test,y_test)

176/176 [==============================] - 1s 3ms/sample - loss: 2.0679 - acc: 0.5114


[2.0679440715096216, 0.5113636]

In [81]:
model.predict_on_batch(X_test)[:10] # looks like the model is overfitting and predicting near 1 (mountain) for everything

array([[0.98168105],
       [0.984094  ],
       [0.9879166 ],
       [0.9836111 ],
       [0.9881889 ],
       [0.98573923],
       [0.98664886],
       [0.9854586 ],
       [0.98731995],
       [0.9878589 ]], dtype=float32)

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [97]:
X.shape

(702, 256, 256, 3)

In [103]:
custom_model = Sequential()
custom_model.add(Conv2D(128, (5,5), activation='relu', input_shape=(256,256,3)))
custom_model.add(MaxPooling2D(4,4))
custom_model.add(Conv2D(128,(5,5), activation='relu'))
custom_model.add(MaxPooling2D(4,4))
custom_model.add(Conv2D(128,(5,5), activation='relu'))
custom_model.add(MaxPooling2D(4,4))
custom_model.add(Flatten())
custom_model.add(Dense(64, activation='relu'))
custom_model.add(Dense(1, activation='sigmoid'))

custom_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 252, 252, 128)     9728      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 63, 63, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 59, 59, 128)       409728    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 10, 10, 128)       409728    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)              

In [0]:
# compile model
custom_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [108]:
# fit model
custom_model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))

Train on 526 samples, validate on 176 samples
Epoch 1/25
526/526 [==============================] - 2s 3ms/sample - loss: nan - acc: 0.5437 - val_loss: nan - val_acc: 0.5114
Epoch 2/25
526/526 [==============================] - 2s 3ms/sample - loss: nan - acc: 0.5399 - val_loss: nan - val_acc: 0.5114
Epoch 3/25
526/526 [==============================] - 2s 3ms/sample - loss: nan - acc: 0.5399 - val_loss: nan - val_acc: 0.5114
Epoch 4/25
526/526 [==============================] - 2s 3ms/sample - loss: nan - acc: 0.5361 - val_loss: nan - val_acc: 0.5114
Epoch 5/25
526/526 [==============================] - 2s 3ms/sample - loss: nan - acc: 0.5361 - val_loss: nan - val_acc: 0.5114
Epoch 6/25
526/526 [==============================] - 2s 3ms/sample - loss: nan - acc: 0.5399 - val_loss: nan - val_acc: 0.5114
Epoch 7/25
526/526 [==============================] - 2s 3ms/sample - loss: nan - acc: 0.5361 - val_loss: nan - val_acc: 0.5114
Epoch 8/25
526/526 [==============================] - 2s 3

### Again, My validation accuracy isn't changing, but my model's accuracy isn't very good anyway

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language